In [98]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [146]:
letters = 'NBR,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z'.split(',')
alphabet_url = 'https://www.metal-archives.com/browse/ajax-letter/json/1/l/'
response_len = 500
payload = {'sEcho': 0, 'iDisplayStart': 0, 'iDisplayLength': response_len}
df_list = []
for letter in letters[:3]:
    r = requests.get(alphabet_url + letter, params=payload)
    js = r.json()
    data = js['aaData']
    df_list.append(pd.DataFrame(data))
df = pd.concat(df_list, axis=0, ignore_index=True)
band_names = df[0].str.split('/').apply(lambda x: x[-3])
df = pd.concat([df, band_names], axis=1, ignore_index=True)
df = df[[4, 1, 2, 0]]
df.columns = ['name', 'country', 'genre', 'link']

In [148]:
df.to_csv('test.csv', index_label=False)

In [18]:
with urlopen('https://www.metal-archives.com/bands/Cynic/91') as page:
    soup = BeautifulSoup(page, 'html.parser')

In [72]:
name = soup.find('h1', attrs={'class': 'band_name'}).text
band_stats_raw = soup.find('div', attrs={'id': 'band_stats'})
keys = band_stats_raw.find_all('dt')
vals = band_stats_raw.find_all('dd')
band_stats = {key.text.replace(':', ''): val.text for key, val in zip(*(keys, vals))}

In [73]:
genre_string = band_stats['Genre']
pattern = '[a-zA-Z]+'
genres = [match.lower() for match in re.findall(pattern, genre_string)]
genres = sorted(set(genres))
if 'metal' in genres:
    genres.remove('metal')
genres

['death', 'early', 'fusion', 'later', 'mid', 'progressive', 'rock', 'thrash']

In [74]:
country = band_stats['Country of origin']
year = band_stats['Formed in']
